In [6]:
"""
AgriGenAI - Day 3: Weather Integration & Hybrid Recommendation Engine
======================================================================
Goal: Integrate real-time weather data and build complete recommendation system
Time: 8 hours

What this does:
1. Set up OpenWeatherMap API integration
2. Fetch real-time weather for farmer's location
3. Build complete recommendation engine with weather scoring
4. Test full pipeline: Image → Traits → Genotype → Weather → Recommendations
5. Generate farmer-friendly output with explanations
"""

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import json
import joblib
import requests
from datetime import datetime
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("🌤️  AgriGenAI - Day 3: Weather Integration")
print("=" * 60)


🌤️  AgriGenAI - Day 3: Weather Integration


In [7]:
# ============================================
# 1. CONFIGURATION
# ============================================

class Config:
    """Configuration for Day 3"""
    # Input paths (from Day 1-2)
    FEATURES_FILE = Path('C:/Users/sahan/Desktop/AgriGenAI2/AgriGenAI_Output/features/phenotype_features.npy')
    METADATA_FILE = Path('C:/Users/sahan/Desktop/AgriGenAI2/AgriGenAI_Output/metadata/image_metadata_with_traits.csv')
    MODELS_PATH = Path('C:/Users/sahan/Desktop/AgriGenAI2/AgriGenAI_Output/models')
    HYBRIDS_PATH = Path('C:/Users/sahan/Desktop/AgriGenAI2/AgriGenAI_Output/hybrids')
    
    # Output paths (Day 3)
    RECOMMENDATIONS_PATH = Path('C:/Users/sahan/Desktop/AgriGenAI2/AgriGenAI_Output/recommendations')
    REPORTS_PATH = Path('C:/Users/sahan/Desktop/AgriGenAI2/AgriGenAI_Output/reports')
    
    # Weather API (Get free key from: https://openweathermap.org/api)
    WEATHER_API_KEY = "2dd75433108cb63b662fef10d29787fa"  # Replace with actual key
    WEATHER_API_URL = "https://api.openweathermap.org/data/2.5/weather"

# Create directories
for path in [Config.RECOMMENDATIONS_PATH, Config.REPORTS_PATH]:
    path.mkdir(parents=True, exist_ok=True)

print("\n✅ Configuration loaded")


✅ Configuration loaded


In [9]:
# ============================================
# 2. LOAD DAY 2 MODELS & DATA
# ============================================

print("\n📂 Loading Day 2 models and data...")

# ⭐ IMPORTANT: Define GenotypePredictorSimulator class BEFORE loading
# (This class was saved in Day 2, so we need its definition to unpickle)

class GenotypePredictorSimulator:
    """
    Simulates genotype prediction from phenotype traits
    In real system, this would use actual genotype-phenotype database
    """
    
    def __init__(self, hybrid_database):
        self.hybrid_db = hybrid_database
        self.genotype_map = self._build_complete_genotype_map()
    
    def _build_complete_genotype_map(self):
        genotype_map = {
            # High Yield Combinations (9 total)
            ('High', 'Resistant', 'High'): {
                'genotype_id': 'G1',
                'genes': ['fw2.2-AA', 'HSP-High', 'Tm-2a-Present'],
                'description': 'Superior genotype: high yield, disease resistant, stress tolerant',
                'breeding_value': 'Excellent - Use as primary parent'
            },
            ('High', 'Resistant', 'Medium'): {
                'genotype_id': 'G2',
                'genes': ['fw2.2-Aa', 'HSP-Medium', 'Tm-2a-Present'],
                'description': 'High yield with disease resistance, moderate stress response',
                'breeding_value': 'Very Good - Suitable for stable climates'
            },
            ('High', 'Resistant', 'Low'): {
                'genotype_id': 'G9',
                'genes': ['fw2.2-AA', 'HSP-Low', 'Tm-2a-Present'],
                'description': 'High yield and disease resistant but stress sensitive',
                'breeding_value': 'Good - Requires controlled environment'
            },
            ('High', 'Moderate', 'High'): {
                'genotype_id': 'G10',
                'genes': ['fw2.2-Aa', 'HSP-High', 'Tm-2-Partial'],
                'description': 'High yield, stress tolerant, moderate disease resistance',
                'breeding_value': 'Very Good - For harsh climates'
            },
            ('High', 'Moderate', 'Medium'): {
                'genotype_id': 'G3',
                'genes': ['fw2.2-Aa', 'HSP-Medium', 'Tm-2-Absent'],
                'description': 'Good all-rounder with high yield potential',
                'breeding_value': 'Good - General purpose variety'
            },
            ('High', 'Moderate', 'Low'): {
                'genotype_id': 'G11',
                'genes': ['fw2.2-AA', 'HSP-Low', 'Tm-2-Absent'],
                'description': 'High yield but vulnerable to disease and stress',
                'breeding_value': 'Fair - Use in protected cultivation'
            },
            ('High', 'Susceptible', 'High'): {
                'genotype_id': 'G12',
                'genes': ['fw2.2-Aa', 'HSP-High', 'Tm-2-Absent'],
                'description': 'High yield and stress tolerant but disease susceptible',
                'breeding_value': 'Fair - Requires disease management'
            },
            ('High', 'Susceptible', 'Medium'): {
                'genotype_id': 'G13',
                'genes': ['fw2.2-AA', 'HSP-Medium', 'Tm-2-Absent'],
                'description': 'High yield but needs disease protection',
                'breeding_value': 'Fair - For experienced farmers'
            },
            ('High', 'Susceptible', 'Low'): {
                'genotype_id': 'G14',
                'genes': ['fw2.2-Aa', 'HSP-Low', 'Tm-2-Absent'],
                'description': 'High yield potential but multiple vulnerabilities',
                'breeding_value': 'Poor - Not recommended for field use'
            },
            
            # Medium Yield Combinations (9 total)
            ('Medium', 'Resistant', 'High'): {
                'genotype_id': 'G5',
                'genes': ['fw2.2-AA', 'HSP-High', 'Tm-2a-Present'],
                'description': 'Balanced genotype: disease resistant, stress tolerant',
                'breeding_value': 'Very Good - Reliable performer'
            },
            ('Medium', 'Resistant', 'Medium'): {
                'genotype_id': 'G6',
                'genes': ['fw2.2-Aa', 'HSP-Medium', 'Tm-2a-Present'],
                'description': 'Moderate yield with good disease resistance',
                'breeding_value': 'Good - Safe choice for farmers'
            },
            ('Medium', 'Resistant', 'Low'): {
                'genotype_id': 'G15',
                'genes': ['fw2.2-aa', 'HSP-Low', 'Tm-2a-Present'],
                'description': 'Disease resistant but stress sensitive, moderate yield',
                'breeding_value': 'Fair - For controlled environments'
            },
            ('Medium', 'Moderate', 'High'): {
                'genotype_id': 'G4',
                'genes': ['fw2.2-aa', 'HSP-High', 'Tm-2-Absent'],
                'description': 'Stress tolerant genotype with moderate yield',
                'breeding_value': 'Good - For marginal lands'
            },
            ('Medium', 'Moderate', 'Medium'): {
                'genotype_id': 'G7',
                'genes': ['fw2.2-Aa', 'HSP-Medium', 'Tm-2-Partial'],
                'description': 'Average genotype across all traits',
                'breeding_value': 'Good - General cultivation variety'
            },
            ('Medium', 'Moderate', 'Low'): {
                'genotype_id': 'G16',
                'genes': ['fw2.2-aa', 'HSP-Low', 'Tm-2-Absent'],
                'description': 'Moderate yield, vulnerable to stress and disease',
                'breeding_value': 'Fair - Requires management'
            },
            ('Medium', 'Susceptible', 'High'): {
                'genotype_id': 'G17',
                'genes': ['fw2.2-Aa', 'HSP-High', 'Tm-2-Absent'],
                'description': 'Stress tolerant but disease susceptible, moderate yield',
                'breeding_value': 'Fair - For dry regions with disease management'
            },
            ('Medium', 'Susceptible', 'Medium'): {
                'genotype_id': 'G18',
                'genes': ['fw2.2-aa', 'HSP-Medium', 'Tm-2-Absent'],
                'description': 'Moderate yield, needs disease protection',
                'breeding_value': 'Fair - For experienced farmers'
            },
            ('Medium', 'Susceptible', 'Low'): {
                'genotype_id': 'G19',
                'genes': ['fw2.2-aa', 'HSP-Low', 'Tm-2-Absent'],
                'description': 'Low breeding value across traits',
                'breeding_value': 'Poor - Not recommended'
            },
            
            # Low Yield Combinations (9 total)
            ('Low', 'Resistant', 'High'): {
                'genotype_id': 'G20',
                'genes': ['fw2.2-aa', 'HSP-High', 'Tm-2a-Present'],
                'description': 'Disease resistant and stress tolerant but low yield',
                'breeding_value': 'Fair - For breeding stock only'
            },
            ('Low', 'Resistant', 'Medium'): {
                'genotype_id': 'G21',
                'genes': ['fw2.2-aa', 'HSP-Medium', 'Tm-2a-Present'],
                'description': 'Disease resistant but low productivity',
                'breeding_value': 'Fair - For conservation breeding'
            },
            ('Low', 'Resistant', 'Low'): {
                'genotype_id': 'G22',
                'genes': ['fw2.2-aa', 'HSP-Low', 'Tm-2a-Present'],
                'description': 'Disease resistant only, poor agronomic traits',
                'breeding_value': 'Poor - Limited use'
            },
            ('Low', 'Moderate', 'High'): {
                'genotype_id': 'G23',
                'genes': ['fw2.2-aa', 'HSP-High', 'Tm-2-Partial'],
                'description': 'Stress tolerant but low yield and moderate disease resistance',
                'breeding_value': 'Fair - For harsh environments only'
            },
            ('Low', 'Moderate', 'Medium'): {
                'genotype_id': 'G24',
                'genes': ['fw2.2-aa', 'HSP-Medium', 'Tm-2-Absent'],
                'description': 'Low productivity, average other traits',
                'breeding_value': 'Poor - Not recommended for cultivation'
            },
            ('Low', 'Moderate', 'Low'): {
                'genotype_id': 'G25',
                'genes': ['fw2.2-aa', 'HSP-Low', 'Tm-2-Absent'],
                'description': 'Poor performance across all traits',
                'breeding_value': 'Very Poor - Avoid'
            },
            ('Low', 'Susceptible', 'High'): {
                'genotype_id': 'G26',
                'genes': ['fw2.2-aa', 'HSP-High', 'Tm-2-Absent'],
                'description': 'Only stress tolerance is acceptable, low yield and disease susceptible',
                'breeding_value': 'Poor - Very limited use'
            },
            ('Low', 'Susceptible', 'Medium'): {
                'genotype_id': 'G27',
                'genes': ['fw2.2-aa', 'HSP-Medium', 'Tm-2-Absent'],
                'description': 'Poor genotype with multiple weaknesses',
                'breeding_value': 'Very Poor - Discard'
            },
            ('Low', 'Susceptible', 'Low'): {
                'genotype_id': 'G8',
                'genes': ['fw2.2-aa', 'HSP-Low', 'Tm-2-Absent'],
                'description': 'Inferior genotype: low yield, disease susceptible, stress sensitive',
                'breeding_value': 'Very Poor - Not viable'
            }
        }
        return genotype_map

    def predict_genotype(self, traits):
        """Predict genotype from trait combination"""
        trait_tuple = (traits['yield'], traits['disease_resistance'], 
                       traits['stress_tolerance'])
        
        # Find matching genotype
        if trait_tuple in self.genotype_map:
            return self.genotype_map[trait_tuple]
        else:
            # Default to closest match
            return {
                'genotype_id': 'G_Unknown',
                'genes': ['Mixed'],
                'description': 'Genotype requires further analysis'
            }
    
    def recommend_hybrids(self, predicted_traits):
        """Recommend hybrid crosses based on predicted traits"""
        recommendations = []
        
        for hybrid_name, hybrid_data in self.hybrid_db.items():
            # Score compatibility (0-100)
            score = 0
            
            # Yield match
            if hybrid_data['traits']['yield'] == predicted_traits['yield']:
                score += 40
            
            # Disease resistance match
            if hybrid_data['traits']['disease_resistance'] == predicted_traits['disease_resistance']:
                score += 30
            
            # Stress tolerance match
            if hybrid_data['traits']['stress_tolerance'] == predicted_traits['stress_tolerance']:
                score += 30
            
            recommendations.append({
                'hybrid_name': hybrid_name,
                'compatibility_score': score,
                'parent_genotypes': hybrid_data['parent_genotypes'],
                'traits': hybrid_data['traits'],
                'genes': hybrid_data['genes']
            })
        
        # Sort by score
        recommendations.sort(key=lambda x: x['compatibility_score'], reverse=True)
        return recommendations[:3]  # Top 3

# NOW load the models (this will work because class is defined)
# Load trained models
models = {}
for model_name in ['yield_trait', 'disease_resistance', 'stress_tolerance']:
    model_file = Config.MODELS_PATH / f'{model_name}_model.pkl'
    models[model_name] = joblib.load(model_file)
    print(f"   ✅ Loaded: {model_name}_model.pkl")

# Load genotype predictor (now it will work!)
genotype_predictor = joblib.load(Config.MODELS_PATH / 'genotype_predictor.pkl')
print(f"   ✅ Loaded: genotype_predictor.pkl")

# Load hybrid database
with open(Config.HYBRIDS_PATH / 'hybrid_database.json', 'r') as f:
    hybrid_database = json.load(f)
print(f"   ✅ Loaded: hybrid_database.json ({len(hybrid_database)} hybrids)")

# Load features and metadata
features = np.load(Config.FEATURES_FILE)
df = pd.read_csv(Config.METADATA_FILE)
print(f"   ✅ Loaded: {features.shape[0]} image features")



📂 Loading Day 2 models and data...
   ✅ Loaded: yield_trait_model.pkl
   ✅ Loaded: disease_resistance_model.pkl
   ✅ Loaded: stress_tolerance_model.pkl
   ✅ Loaded: genotype_predictor.pkl
   ✅ Loaded: hybrid_database.json (5 hybrids)
   ✅ Loaded: 15313 image features


In [15]:
# ============================================
# 3. WEATHER SERVICE
# ============================================

print("\n🌤️  Setting up weather service...")

class WeatherService:
    """Fetch real-time weather data"""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://api.openweathermap.org/data/2.5/weather"
    
    def get_weather(self, location: str = "Mysuru,IN") -> Dict:
        """
        Get current weather for location
        
        Args:
            location: City,CountryCode (e.g., "Mysuru,IN")
        
        Returns:
            Dict with temperature, humidity, conditions
        """
        # Check if API key is set
        if self.api_key == "YOUR_API_KEY_HERE":
            print("   ⚠️  Using fallback weather data (API key not configured)")
            return self._get_fallback_weather(location)
        
        try:
            params = {
                'q': location,
                'appid': self.api_key,
                'units': 'metric'  # Celsius
            }
            
            response = requests.get(self.base_url, params=params, timeout=5)
            
            if response.status_code == 200:
                data = response.json()
                return {
                    'location': location,
                    'temperature': data['main']['temp'],
                    'humidity': data['main']['humidity'],
                    'pressure': data['main']['pressure'],
                    'weather': data['weather'][0]['main'],
                    'description': data['weather'][0]['description'],
                    'wind_speed': data['wind']['speed'],
                    'timestamp': datetime.now().isoformat(),
                    'source': 'OpenWeatherMap API'
                }
            else:
                print(f"   ⚠️  Weather API error: {response.status_code}")
                return self._get_fallback_weather(location)
        
        except Exception as e:
            print(f"   ⚠️  Weather fetch failed: {e}")
            return self._get_fallback_weather(location)
    
    def _get_fallback_weather(self, location: str) -> Dict:
        """Fallback weather data for demo"""
        return {
            'location': location,
            'temperature': 32.0,
            'humidity': 65,
            'pressure': 1012,
            'weather': 'Clear',
            'description': 'clear sky',
            'wind_speed': 3.5,
            'timestamp': datetime.now().isoformat(),
            'source': 'Fallback (typical Mysuru summer)'
        }

# Initialize weather service
weather_service = WeatherService(Config.WEATHER_API_KEY)
print("   ✅ Weather service initialized")


🌤️  Setting up weather service...
   ✅ Weather service initialized


In [16]:
# ============================================
# 4. HYBRID RECOMMENDATION ENGINE
# ============================================

print("\n🌱 Building hybrid recommendation engine...")

class HybridRecommendationEngine:
    """Complete recommendation system with weather integration"""
    
    def __init__(self, hybrid_db: Dict, weather_service: WeatherService):
        self.hybrid_db = hybrid_db
        self.weather_service = weather_service
    
    def calculate_trait_score(
        self, 
        predicted_traits: Dict, 
        hybrid_traits: Dict
    ) -> Tuple[int, List[str]]:
        """Calculate trait compatibility score (0-100)"""
        score = 0
        reasons = []
        
        # Yield compatibility (40 points)
        if predicted_traits.get('yield') == hybrid_traits.get('yield'):
            score += 40
            reasons.append(f"✅ Yield match: {hybrid_traits['yield']}")
        elif self._is_adjacent(
            predicted_traits.get('yield'), 
            hybrid_traits.get('yield'),
            ['Low', 'Medium', 'High']
        ):
            score += 20
            reasons.append(f"⚠️ Partial yield match ({predicted_traits.get('yield')} → {hybrid_traits['yield']})")
        
        # Disease resistance compatibility (30 points)
        if predicted_traits.get('disease_resistance') == hybrid_traits.get('disease_resistance'):
            score += 30
            reasons.append(f"✅ Disease resistance match: {hybrid_traits['disease_resistance']}")
        elif self._is_adjacent(
            predicted_traits.get('disease_resistance'),
            hybrid_traits.get('disease_resistance'),
            ['Susceptible', 'Moderate', 'Resistant']
        ):
            score += 15
            reasons.append(f"⚠️ Partial resistance match")
        
        # Stress tolerance compatibility (30 points)
        if predicted_traits.get('stress_tolerance') == hybrid_traits.get('stress_tolerance'):
            score += 30
            reasons.append(f"✅ Stress tolerance match: {hybrid_traits['stress_tolerance']}")
        elif self._is_adjacent(
            predicted_traits.get('stress_tolerance'),
            hybrid_traits.get('stress_tolerance'),
            ['Low', 'Medium', 'High']
        ):
            score += 15
            reasons.append(f"⚠️ Partial tolerance match")
        
        return score, reasons
    
    def calculate_weather_score(
        self, 
        hybrid_data: Dict, 
        weather: Dict
    ) -> Tuple[int, List[str]]:
        """Calculate weather compatibility bonus (0-20 points)"""
        score = 0
        reasons = []
        
        temp = weather['temperature']
        humidity = weather['humidity']
        
        # Temperature compatibility (10 points)
        temp_min, temp_max = hybrid_data['optimal_temp']
        if temp_min <= temp <= temp_max:
            score += 10
            reasons.append(f"✅ Temperature perfect: {temp}°C (optimal: {temp_min}-{temp_max}°C)")
        else:
            temp_diff = min(abs(temp - temp_min), abs(temp - temp_max))
            if temp_diff <= 5:
                score += 5
                reasons.append(f"⚠️ Temperature acceptable: {temp}°C ({temp_diff}°C from optimal)")
            else:
                reasons.append(f"❌ Temperature suboptimal: {temp}°C (optimal: {temp_min}-{temp_max}°C)")
        
        # Humidity compatibility (10 points)
        hum_min, hum_max = hybrid_data['humidity_tolerance']
        if hum_min <= humidity <= hum_max:
            score += 10
            reasons.append(f"✅ Humidity perfect: {humidity}% (optimal: {hum_min}-{hum_max}%)")
        else:
            hum_diff = min(abs(humidity - hum_min), abs(humidity - hum_max))
            if hum_diff <= 10:
                score += 5
                reasons.append(f"⚠️ Humidity acceptable: {humidity}% ({hum_diff}% from optimal)")
            else:
                reasons.append(f"❌ Humidity suboptimal: {humidity}% (optimal: {hum_min}-{hum_max}%)")
        
        return score, reasons
    
    def recommend_hybrids(
        self,
        predicted_traits: Dict,
        location: str = "Mysuru,IN",
        top_n: int = 3
    ) -> Dict:
        """
        Main recommendation function
        
        Returns complete recommendation report
        """
        # Fetch weather
        weather = self.weather_service.get_weather(location)
        
        recommendations = []
        
        for hybrid_name, hybrid_data in self.hybrid_db.items():
            # Calculate trait compatibility
            trait_score, trait_reasons = self.calculate_trait_score(
                predicted_traits,
                hybrid_data['traits']
            )
            
            # Calculate weather compatibility
            weather_score, weather_reasons = self.calculate_weather_score(
                hybrid_data,
                weather
            )
            
            # Total score (max 120, normalized to 100)
            total_score = min(100, trait_score + weather_score)
            
            recommendations.append({
                'rank': 0,  # Will be set after sorting
                'hybrid_name': hybrid_name,
                'total_score': total_score,
                'trait_score': trait_score,
                'weather_score': weather_score,
                'parent_genotypes': hybrid_data['parent_genotypes'],
                'traits': hybrid_data['traits'],
                'genes': hybrid_data['genes'],
                'maturity_days': hybrid_data['traits']['maturity_days'],
                'fruit_size': hybrid_data['traits']['fruit_size'],
                'optimal_temp': hybrid_data['optimal_temp'],
                'humidity_tolerance': hybrid_data['humidity_tolerance'],
                'trait_reasons': trait_reasons,
                'weather_reasons': weather_reasons
            })
        
        # Sort by total score
        recommendations.sort(key=lambda x: x['total_score'], reverse=True)
        
        # Assign ranks
        for i, rec in enumerate(recommendations):
            rec['rank'] = i + 1
        
        return {
            'weather': weather,
            'predicted_traits': predicted_traits,
            'recommendations': recommendations[:top_n],
            'all_recommendations': recommendations,
            'timestamp': datetime.now().isoformat()
        }
    
    def _is_adjacent(self, val1: str, val2: str, ordered_list: List[str]) -> bool:
        """Check if two values are adjacent in ordered list"""
        try:
            idx1 = ordered_list.index(val1)
            idx2 = ordered_list.index(val2)
            return abs(idx1 - idx2) == 1
        except (ValueError, AttributeError):
            return False

# Initialize recommendation engine
recommendation_engine = HybridRecommendationEngine(hybrid_database, weather_service)
print("   ✅ Recommendation engine ready")



🌱 Building hybrid recommendation engine...
   ✅ Recommendation engine ready


In [13]:
# ============================================
# 5. COMPLETE PREDICTION PIPELINE
# ============================================

print("\n🔄 Building complete prediction pipeline...")

def predict_and_recommend(
    image_idx: int,
    location: str = "Mysuru,IN"
) -> Dict:
    """
    Complete pipeline: Image → Traits → Genotype → Recommendations
    
    Args:
        image_idx: Index of image in dataset
        location: Location for weather data
    
    Returns:
        Complete recommendation report
    """
    # Get image info
    image_info = df.iloc[image_idx]
    
    # Extract features
    sample_features = features[image_idx].reshape(1, -1)
    
    # Predict traits
    predicted_traits = {}
    for trait_name, model in models.items():
        pred = model.predict(sample_features)[0]
        predicted_traits[trait_name.replace('_trait', '')] = pred
    
    # Predict genotype
    genotype = genotype_predictor.predict_genotype(predicted_traits)
    
    # Get hybrid recommendations
    recommendations = recommendation_engine.recommend_hybrids(
        predicted_traits,
        location
    )
    
    # Compile complete report
    report = {
        'image_info': {
            'path': image_info['image_path'],
            'category': image_info['category'],
            'organ': image_info['organ'],
            'source': image_info['source']
        },
        'predicted_traits': predicted_traits,
        'predicted_genotype': genotype,
        'weather': recommendations['weather'],
        'recommendations': recommendations['recommendations'],
        'timestamp': recommendations['timestamp']
    }
    
    return report

print("   ✅ Complete pipeline ready")


🔄 Building complete prediction pipeline...
   ✅ Complete pipeline ready


In [17]:
# ============================================
# 6. TEST COMPLETE SYSTEM
# ============================================

print("\n🧪 Testing complete system with real examples...")

# Test with different scenarios
test_cases = [
    {'idx': 0, 'location': 'Mysuru,IN', 'desc': 'Bacterial spot leaf'},
    {'idx': 1000, 'location': 'Bangalore,IN', 'desc': 'Healthy leaf'},
    {'idx': len(df)-1, 'location': 'Mysuru,IN', 'desc': 'Fruit image'}
]

test_results = []

for test_case in test_cases:
    print(f"\n{'='*60}")
    print(f"Test Case: {test_case['desc']}")
    print(f"Location: {test_case['location']}")
    print('='*60)
    
    result = predict_and_recommend(test_case['idx'], test_case['location'])
    test_results.append(result)
    
    # Display results
    print(f"\n📸 Image: {result['image_info']['path']}")
    print(f"   Category: {result['image_info']['category']}")
    
    print(f"\n🔮 Predicted Traits:")
    for trait, value in result['predicted_traits'].items():
        print(f"   {trait.title()}: {value}")
    
    print(f"\n🧬 Predicted Genotype:")
    print(f"   ID: {result['predicted_genotype']['genotype_id']}")
    print(f"   Genes: {', '.join(result['predicted_genotype']['genes'])}")
    
    print(f"\n🌤️  Weather ({result['weather']['location']}):")
    print(f"   Temperature: {result['weather']['temperature']}°C")
    print(f"   Humidity: {result['weather']['humidity']}%")
    print(f"   Conditions: {result['weather']['description']}")
    
    print(f"\n🏆 Top 3 Hybrid Recommendations:")
    for rec in result['recommendations']:
        print(f"\n   #{rec['rank']}: {rec['hybrid_name']} (Score: {rec['total_score']}/100)")
        print(f"   Parents: {' × '.join(rec['parent_genotypes'])}")
        print(f"   Expected: {rec['traits']['yield']} yield, {rec['maturity_days']} days")
        print(f"   Trait Score: {rec['trait_score']}/100 | Weather Score: {rec['weather_score']}/20")
        print(f"   Why this cross:")
        for reason in rec['trait_reasons'][:2]:  # Show top 2 reasons
            print(f"      {reason}")

print(f"\n{'='*60}")
print("✅ All test cases completed successfully!")
print('='*60)


🧪 Testing complete system with real examples...

Test Case: Bacterial spot leaf
Location: Mysuru,IN

📸 Image: ..\AgriGenAI_Dataset\PlantVillage\images\Bacterial_spot\Bs10.jpg
   Category: Bacterial_spot

🔮 Predicted Traits:
   Yield: Medium
   Disease_Resistance: Moderate
   Stress_Tolerance: Medium

🧬 Predicted Genotype:
   ID: G7
   Genes: fw2.2-Aa, HSP-Medium, Tm-2-Partial

🌤️  Weather (Mysuru,IN):
   Temperature: 20.04°C
   Humidity: 94%
   Conditions: overcast clouds

🏆 Top 3 Hybrid Recommendations:

   #1: Punjab_Chhuhara (Score: 100/100)
   Parents: G4 × G8
   Expected: Medium yield, 60 days
   Trait Score: 85/100 | Weather Score: 15/20
   Why this cross:
      ✅ Yield match: Medium
      ✅ Disease resistance match: Moderate

   #2: Pusa_Ruby (Score: 90/100)
   Parents: G3 × G4
   Expected: High yield, 65 days
   Trait Score: 80/100 | Weather Score: 10/20
   Why this cross:
      ⚠️ Partial yield match (Medium → High)
      ✅ Disease resistance match: Moderate

   #3: Himsona (

In [18]:
# ============================================
# 7. GENERATE FARMER-FRIENDLY REPORT
# ============================================

print("\n📄 Generating farmer-friendly report...")

def generate_farmer_report(result: Dict) -> str:
    """Generate plain-language report for farmers"""
    
    weather = result['weather']
    traits = result['predicted_traits']
    genotype = result['predicted_genotype']
    recs = result['recommendations']
    
    report = f"""
╔══════════════════════════════════════════════════════════════╗
║           AgriGenAI - Hybrid Recommendation Report          ║
╚══════════════════════════════════════════════════════════════╝

📅 Date: {datetime.now().strftime('%d %B %Y, %I:%M %p')}
📍 Location: {weather['location']}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🌤️  CURRENT WEATHER CONDITIONS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Temperature:  {weather['temperature']}°C
Humidity:     {weather['humidity']}%
Conditions:   {weather['description'].title()}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🌱 YOUR PLANT ANALYSIS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Yield Potential:      {traits['yield']}
Disease Resistance:   {traits['disease_resistance']}
Stress Tolerance:     {traits['stress_tolerance']}

Genetic Profile:      {genotype['genotype_id']}
Key Genes:            {', '.join(genotype['genes'])}

🔍 What this means:
   {genotype['description']}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🏆 TOP 3 RECOMMENDED HYBRID CROSSES
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""
    
    for rec in recs:
        report += f"""
┌──────────────────────────────────────────────────────────────┐
│ #{rec['rank']} RECOMMENDATION: {rec['hybrid_name'].upper()}
│ Match Score: {rec['total_score']}/100 {'⭐' * (rec['total_score'] // 20)}
└──────────────────────────────────────────────────────────────┘

🧬 Parent Cross:      {' × '.join(rec['parent_genotypes'])}
🌾 Expected Yield:    {rec['traits']['yield']}
🍅 Fruit Size:        {rec['fruit_size']}
⏱️  Days to Maturity:  {rec['maturity_days']} days
🌡️  Temperature Range: {rec['optimal_temp'][0]}-{rec['optimal_temp'][1]}°C
💧 Humidity Range:    {rec['humidity_tolerance'][0]}-{rec['humidity_tolerance'][1]}%

✅ Why This Cross?
"""
        for reason in (rec['trait_reasons'] + rec['weather_reasons'])[:4]:
            report += f"   • {reason}\n"
        
        report += "\n"
    
    report += """
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
💡 NEXT STEPS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Review the top recommendations above
2. Consider your local market demand and budget
3. Purchase seeds from certified dealers
4. Follow recommended planting dates for your region

📞 For more information, contact your local agricultural extension office.

═══════════════════════════════════════════════════════════════
Generated by AgriGenAI | Powered by AI & Real-Time Weather Data
═══════════════════════════════════════════════════════════════
"""
    
    return report

# Generate reports for test cases
for i, result in enumerate(test_results):
    report_text = generate_farmer_report(result)
    
    # Save report
    report_file = Config.REPORTS_PATH / f'recommendation_report_{i+1}.txt'
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write(report_text)
    
    print(f"   ✅ Saved: {report_file.name}")

# Display one sample report
print("\n" + "="*60)
print("📄 SAMPLE FARMER REPORT")
print("="*60)
print(generate_farmer_report(test_results[0]))



📄 Generating farmer-friendly report...
   ✅ Saved: recommendation_report_1.txt
   ✅ Saved: recommendation_report_2.txt
   ✅ Saved: recommendation_report_3.txt

📄 SAMPLE FARMER REPORT

╔══════════════════════════════════════════════════════════════╗
║           AgriGenAI - Hybrid Recommendation Report          ║
╚══════════════════════════════════════════════════════════════╝

📅 Date: 23 October 2025, 08:45 PM
📍 Location: Mysuru,IN

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🌤️  CURRENT WEATHER CONDITIONS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Temperature:  20.04°C
Humidity:     94%
Conditions:   Overcast Clouds

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🌱 YOUR PLANT ANALYSIS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Yield Potential:      Medium
Disease Resistance:   Moderate
Stress Tolerance:     Medium

Genetic Profile:      G7
Key Genes:            fw2.2-Aa, HSP-Medium, Tm-2-Partial

🔍 What this 

In [19]:
# ============================================
# 8. SAVE RECOMMENDATION ENGINE
# ============================================

print("\n💾 Saving recommendation engine...")

# Save recommendation engine
engine_file = Config.MODELS_PATH / 'recommendation_engine.pkl'
joblib.dump(recommendation_engine, engine_file)
print(f"   ✅ Saved: {engine_file}")


💾 Saving recommendation engine...
   ✅ Saved: C:\Users\sahan\Desktop\AgriGenAI2\AgriGenAI_Output\models\recommendation_engine.pkl


In [20]:
# ============================================
# 9. SUMMARY
# ============================================

print("\n" + "=" * 60)
print("✅ DAY 3 COMPLETE!")
print("=" * 60)

print(f"\n📊 Summary:")
print(f"   - Weather integration: ✅ Working")
print(f"   - Recommendation engine: ✅ Complete")
print(f"   - Test cases executed: {len(test_results)}")
print(f"   - Reports generated: {len(test_results)}")

print(f"\n📁 Outputs saved:")
print(f"   - Recommendation engine: {engine_file}")
print(f"   - Farmer reports: {Config.REPORTS_PATH}")

print(f"\n🎯 System Capabilities:")
print(f"   ✅ Image-based trait prediction (93% avg accuracy)")
print(f"   ✅ Genotype inference from traits")
print(f"   ✅ Real-time weather integration")
print(f"   ✅ Multi-factor hybrid scoring")
print(f"   ✅ Farmer-friendly recommendations")

print(f"\n🚀 Ready for Day 4: Backend API Development!")
print("=" * 60)


✅ DAY 3 COMPLETE!

📊 Summary:
   - Weather integration: ✅ Working
   - Recommendation engine: ✅ Complete
   - Test cases executed: 3
   - Reports generated: 3

📁 Outputs saved:
   - Recommendation engine: C:\Users\sahan\Desktop\AgriGenAI2\AgriGenAI_Output\models\recommendation_engine.pkl
   - Farmer reports: C:\Users\sahan\Desktop\AgriGenAI2\AgriGenAI_Output\reports

🎯 System Capabilities:
   ✅ Image-based trait prediction (93% avg accuracy)
   ✅ Genotype inference from traits
   ✅ Real-time weather integration
   ✅ Multi-factor hybrid scoring
   ✅ Farmer-friendly recommendations

🚀 Ready for Day 4: Backend API Development!
